In [ ]:
!pip install ktrain

     |████████████████████████████████| 25.3 MB 90 kB/s 
     |████████████████████████████████| 6.8 MB 39.1 MB/s 
     |████████████████████████████████| 981 kB 37.3 MB/s 
     |████████████████████████████████| 263 kB 42.6 MB/s 
     |████████████████████████████████| 1.9 MB 37.3 MB/s 
     |████████████████████████████████| 1.2 MB 35.4 MB/s 
     |████████████████████████████████| 468 kB 33.3 MB/s 
     |████████████████████████████████| 3.3 MB 31.5 MB/s 
     |████████████████████████████████| 895 kB 31.5 MB/s 
  Created wheel for ktrain: filename=ktrain-0.27.3-py3-none-any.whl size=25283131 sha256=c57c97431738f3aa12df63070014242637fe98286f3f29e6f3844c529b52487e
  Stored in directory: /root/.cache/pip/wheels/3c/ef/e1/7da805d8a5944e8a3ac0553831d832c00b1800b848939849cb
  Created wheel for seqeval: filename=seqeval-0.0.19-py3-none-any.whl size=9929 sha256=6df355e41a02d275b3571209133f6383bee1276866e2241535d5d673126dc0e9
  Stored in directory: /root/.cache/pip/wheels/f5/ac/f1/4e13d7aff0

In [ ]:
import warnings
warnings.simplefilter('ignore')

!git clone https://github.com/laxmimerit/IMDB-Movie-Reviews-Large-Dataset-50k.git

Cloning into 'IMDB-Movie-Reviews-Large-Dataset-50k'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 10 (delta 1), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (10/10), done.


In [ ]:
import numpy as np 
import pandas as pd
import tensorflow as tf
import ktrain
from ktrain import text

In [ ]:
train=pd.read_excel('/content/IMDB-Movie-Reviews-Large-Dataset-50k/train.xlsx', dtype=str)
test=pd.read_excel('/content/IMDB-Movie-Reviews-Large-Dataset-50k/test.xlsx', dtype=str)
train.head(5)

,Reviews,Sentiment
0,"When I first tuned in on this morning news, I ...",neg
1,"Mere thoughts of ""Going Overboard"" (aka ""Babes...",neg
2,Why does this movie fall WELL below standards?...,neg
3,Wow and I thought that any Steven Segal movie ...,neg
4,"The story is seen before, but that does'n matt...",neg


In [ ]:
(x_train, y_train), (x_test, y_test), preprocess = text.texts_from_df(train_df=train, 
                            text_column='Reviews',
                            label_columns='Sentiment', 
                            val_df=test,
                            maxlen=400, 
                            preprocess_mode='bert')

['neg', 'pos']
   neg  pos
0  1.0  0.0
1  1.0  0.0
2  1.0  0.0
3  1.0  0.0
4  1.0  0.0
['neg', 'pos']
   neg  pos
0  0.0  1.0
1  0.0  1.0
2  1.0  0.0
3  0.0  1.0
4  1.0  0.0
preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


In [ ]:
x_train[0].shape

(25000, 400)

In [ ]:
model=text.text_classifier(name='bert',
                           train_data=(x_train, y_train),
                           preproc=preprocess)

Is Multi-Label? False
maxlen is 400
done.


In [ ]:
# get learning rate
learner=ktrain.get_learner(model=model,
                           train_data=(x_train, y_train),
                           val_data=(x_test, y_test),
                           batch_size=6)

In [ ]:
# optimal learning rate
# takes days
#learner.lr_find()
#learner.lr_plot

#optimal learner rate for the model is 2e-5

In [ ]:
# fitting algorithm
#learner.fit_onecycle(lr=2e-5, epochs=1)

In [ ]:
predictor=ktrain.get_predictor(learner.model, preprocess)

In [ ]:
data=['this movie was horrible. the plot was really boring. Acting wasokay. though',
      'this file really sucks. i want my money back',
      'what a beautiful movie. great plot, great acting. will see it again']

In [ ]:
predictor.predict(data)

['neg', 'neg', 'pos']

In [ ]:
predictor.save('/content/bert')